## **Note!**

Make sure you install pygame in your terminal: \
pip install pygame

## **Prototype 4 - Expanded Environment**

We expand the environment horizontally. In here we add cooldown for adding and removing block to mitigate cheating or excessive clicking.

**To test it, run the code**

In [ ]:
import pygame
import sys
import random
import heapq
from datetime import datetime, timedelta

# Constants
WIDTH, HEIGHT = 800, 600
CELL_SIZE = 20
COLS = 100  # Expanded horizontally
ROWS = HEIGHT // CELL_SIZE
AGENT_SPEED = 200  # pixels per second
MIN_DISTANCE = COLS // 4  # Minimum distance between start and end

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)
GREY = (200, 200, 200)

# Camera control
camera_x = 0
CAMERA_LERP = 0.1
CAMERA_OFFSET = WIDTH // 4  # Agent appears 25% from left
MAX_CAMERA_X = (COLS * CELL_SIZE) - WIDTH

class SmoothAgent:
    def __init__(self, start_pos):
        self.target_pos = start_pos
        self.current_pos = pygame.Vector2(
            start_pos[1] * CELL_SIZE + CELL_SIZE/2,
            start_pos[0] * CELL_SIZE + CELL_SIZE/2
        )
        self.speed = AGENT_SPEED
        self.moving = False

    def update(self, path, dt):
        if not path or self.current_pos == self.target_pos:
            return

        target_x = self.target_pos[1] * CELL_SIZE + CELL_SIZE/2
        target_y = self.target_pos[0] * CELL_SIZE + CELL_SIZE/2

        direction = pygame.Vector2(target_x - self.current_pos.x,
                                 target_y - self.current_pos.y)
        if direction.length() > 0:
            direction.scale_to_length(self.speed * dt)
            self.current_pos += direction

            if (self.current_pos - pygame.Vector2(target_x, target_y)).length() < 2:
                self.current_pos = pygame.Vector2(target_x, target_y)

def generate_maze(rows, cols):
    # Ensure start and end positions meet distance requirement
    while True:
        maze = [[1 for _ in range(cols)] for _ in range(rows)]
        start = (random.randint(0, rows-1), random.randint(0, cols//8))
        end = (random.randint(0, rows-1), random.randint(cols - cols//8, cols-1))

        if abs(start[1] - end[1]) < MIN_DISTANCE:
            continue

        stack = [start]
        maze[start[0]][start[1]] = 0
        directions = [(-1,0),(1,0),(0,-1),(0,1)]

        while stack:
            current = stack[-1]
            neighbors = []
            for dx, dy in directions:
                nx, ny = current[0]+dx*2, current[1]+dy*2
                if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 1:
                    neighbors.append((nx, ny))

            if neighbors:
                next_cell = random.choice(neighbors)
                wall = (current[0]+(next_cell[0]-current[0])//2,
                        current[1]+(next_cell[1]-current[1])//2)
                maze[wall[0]][wall[1]] = 0
                maze[next_cell[0]][next_cell[1]] = 0
                stack.append(next_cell)
            else:
                stack.pop()

        # Add horizontal paths
        for _ in range(cols * 2):
            x, y = random.randint(0, rows-1), random.randint(0, cols-1)
            maze[x][y] = 0

        if astar(maze, start, end):
            return maze, start, end

# A* Pathfinding Algorithm
def astar(maze, start, end):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = { (r, c): float('inf') for r in range(ROWS) for c in range(COLS) }
    g_score[start] = 0
    f_score = { (r, c): float('inf') for r in range(ROWS) for c in range(COLS) }
    f_score[start] = heuristic(start, end)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == end:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(maze, current):
            tentative_g = g_score[current] + 1
            if tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, end)
                if neighbor not in [i[1] for i in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(maze, pos):
    neighbors = []
    for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        nx, ny = pos[0]+dx, pos[1]+dy
        if 0 <= nx < ROWS and 0 <= ny < COLS:
            if maze[nx][ny] == 0:
                neighbors.append((nx, ny))
    return neighbors

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

def draw_text(surface, text, pos, size=30, color=BLACK):
    font = pygame.font.SysFont(None, size)
    text_surface = font.render(text, True, color)
    surface.blit(text_surface, pos)

def update_camera(dt):
    global camera_x
    target_x = agent.current_pos.x - CAMERA_OFFSET
    target_x = max(0, min(target_x, MAX_CAMERA_X))
    camera_x += (target_x - camera_x) * CAMERA_LERP * dt * 60

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mega Maze Pathfinder")

# Generate initial maze
maze, start, end = generate_maze(ROWS, COLS)
original_maze = [row[:] for row in maze]
path = astar(maze, start, end)
agent = SmoothAgent(start)
game_over = False
show_path = True
last_block_time = datetime.now()
block_cooldown = 2  # seconds
path_index = 0
clock = pygame.time.Clock()

while True:
    dt = clock.tick(60)/1000
    time_since_last_block = datetime.now() - last_block_time
    can_place_block = time_since_last_block.total_seconds() >= block_cooldown

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over and can_place_block:
            mx, my = pygame.mouse.get_pos()
            world_x = mx + camera_x
            col = int(world_x // CELL_SIZE)
            row = int(my // CELL_SIZE)

            if 0 <= row < ROWS and 0 <= col < COLS:
                if event.button == 1:  # Add block
                    if maze[row][col] == 0 and (row, col) != start and (row, col) != end:
                        maze[row][col] = 1
                        last_block_time = datetime.now()
                elif event.button == 3:  # Remove block
                    if maze[row][col] == 1:
                        maze[row][col] = 0
                        last_block_time = datetime.now()

                # Recalculate path
                current_grid = (
                    int(agent.current_pos.y // CELL_SIZE),
                    int(agent.current_pos.x // CELL_SIZE)
                )
                new_path = astar(maze, current_grid, end)
                if new_path:
                    path = new_path
                    path_index = 0
                    agent.target_pos = path[path_index]
                else:
                    game_over = True

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:  # Reset current maze
                maze = [row[:] for row in original_maze]
                path = astar(maze, start, end)
                game_over = False
                path_index = 0
                agent = SmoothAgent(start)
                camera_x = 0
            elif event.key == pygame.K_t:  # New maze
                maze, start, end = generate_maze(ROWS, COLS)
                original_maze = [row[:] for row in maze]
                path = astar(maze, start, end)
                game_over = False
                path_index = 0
                agent = SmoothAgent(start)
                camera_x = 0
            elif event.key == pygame.K_p:
                show_path = not show_path

    # Agent movement
    if path and not game_over and path_index < len(path):
        agent.target_pos = path[path_index]
        if agent.current_pos.distance_to(
            (agent.target_pos[1]*CELL_SIZE + CELL_SIZE/2,
             agent.target_pos[0]*CELL_SIZE + CELL_SIZE/2)
        ) < 2:
            path_index += 1

    agent.update(path, dt)
    update_camera(dt)

    # Drawing
    screen.fill(WHITE)

    # Visible columns
    start_col = int(camera_x // CELL_SIZE)
    end_col = start_col + (WIDTH // CELL_SIZE) + 2

    # Draw maze
    for r in range(ROWS):
        for c in range(max(0, start_col), min(COLS, end_col)):
            if maze[r][c] == 1:
                pygame.draw.rect(screen, BLACK,
                               (c*CELL_SIZE - camera_x, r*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Start/End points
    pygame.draw.rect(screen, GREEN,
                   (start[1]*CELL_SIZE - camera_x, start[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))
    pygame.draw.rect(screen, BLUE,
                   (end[1]*CELL_SIZE - camera_x, end[0]*CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Projected path
    if show_path and path and not game_over and path_index < len(path):
        for p in path[path_index:]:
            pygame.draw.circle(screen, ORANGE,
                             (p[1]*CELL_SIZE - camera_x + CELL_SIZE//2,
                              p[0]*CELL_SIZE + CELL_SIZE//2),
                             2)

    # Agent
    pygame.draw.circle(screen, RED,
                     (agent.current_pos.x - camera_x,
                      agent.current_pos.y),
                     CELL_SIZE//3)

    # UI
    if not can_place_block:
        remaining = block_cooldown - time_since_last_block.total_seconds()
        draw_text(screen, f"Block Cooldown: {remaining:.1f}s", (10, HEIGHT-40), 30, RED)
    else:
        draw_text(screen, "Ready to place blocks", (10, HEIGHT-40), 30, GREEN)

    draw_text(screen, f"Path Visible: {'ON' if show_path else 'OFF'} (Press P)", (10, 10), 30)

    if game_over:
        draw_text(screen, "All paths blocked!", (WIDTH//2-100, HEIGHT//2-20), 40, RED)
        draw_text(screen, "R: Restart  T: New Maze", (WIDTH//2-120, HEIGHT//2+20), 30)

    pygame.display.flip()